<a href="https://colab.research.google.com/github/melinnicri/Scrapeo_web/blob/main/TareaAmeliaCris.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Tarea final de Wescraping con Python:**
- Amelia Cristina Herrera Briceño
- melinnicri@gmail.com


1.- Realice el Web scraping de la siguiente página usando selenium “https://subslikescript.com/” cree un dataframe (en *.csv) en donde extraiga las películas y sus correspondientes links. (6 puntos).


In [95]:
# Instalar selenium
!pip install selenium

In [96]:
# Instalar chromium- chromedriver
!apt install chromium-chromedriver

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
chromium-chromedriver is already the newest version (1:85.0.4183.83-0ubuntu2.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 44 not upgraded.


In [110]:
# Importar librerías
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from time import sleep
import pandas as pd

from selenium.common.exceptions import WebDriverException


# para eliminar warning en python
import warnings
warnings.simplefilter('ignore')

In [111]:
# Configuración del webdriver
def web_driver():
    optiones = webdriver.ChromeOptions()
    optiones.add_argument("--verbose")
    optiones.add_argument('--no-sandbox')
    optiones.add_argument('--headless')
    optiones.add_argument('--disable-gpu')
    optiones.add_argument("--window-size=1920, 1200")
    optiones.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options = optiones)
    return driver


In [112]:
# Inicializamos el driver, se maneja con elegancia cualquier excepción que pueda ocurrir durante el proceso.
# Se cierra la ventana del navegador y abandona la instancia de WebDriver.
def get_movie_data():
    try:
        driver = web_driver()
        driver.get("https://subslikescript.com/movies")

    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        driver.quit()

if __name__ == "__main__":
    get_movie_data()


In [113]:
# Extraer los datos de las películas de la página 1 (esto es un ensayo)
def extract_movie_data(driver):
    movie_list = []
    movie_elements = driver.find_elements(By.CSS_SELECTOR, "ul.scripts-list li a")
    for movie_element in movie_elements:
        href = movie_element.get_attribute("href")
        title = movie_element.get_attribute("title")
        movie_title = movie_element.text.strip()
        movie_data = {
            "href": href,
            "title": title,
            "movie_title": movie_title
        }
        movie_list.append(movie_data)
    return movie_list

def scrape_all_pages(driver, base_url):
    all_movie_data = []
    page_number = 1
    while True:
        url = f"{base_url}?page={page_number}"
        driver.get(url)

        # Esperar que la página cargue 10 seg
        wait = WebDriverWait(driver, 10)
        wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "ul.scripts-list li a")))

        movie_data = extract_movie_data(driver)
        all_movie_data.extend(movie_data)

        # Encontrar el próximo elemento de la página
        next_page_element = driver.find_element(By.CSS_SELECTOR, ".pagination_pages .page-item:last-child a")
        if "next" not in next_page_element.get_attribute("aria-label"):
            break  # No más páginas

        page_number += 1

    return all_movie_data

# Ejemplo de uso:
driver = web_driver()  # Reemplazar con el WebDriver deseado
base_url = "https://subslikescript.com/movies"

all_movies = scrape_all_pages(driver, base_url)
print(all_movies)

driver.quit()


[{'href': 'https://subslikescript.com/movie/Vikingdom-1785669', 'title': "Read transcript of Movie 'Vikingdom'", 'movie_title': 'Vikingdom (2013)'}, {'href': 'https://subslikescript.com/movie/LEGO_Disney_Princess_The_Castle_Quest-28477869', 'title': "Read transcript of Movie 'LEGO Disney Princess: The Castle Quest'", 'movie_title': 'LEGO Disney Princess: The Castle Quest (2023)'}, {'href': 'https://subslikescript.com/movie/Atrangi_Re-9766332', 'title': "Read transcript of Movie 'Atrangi Re'", 'movie_title': 'Atrangi Re (2021)'}, {'href': 'https://subslikescript.com/movie/Love_Sex_and_30_Candles-28508739', 'title': "Read transcript of Movie 'Love, Sex and 30 Candles'", 'movie_title': 'Love, Sex and 30 Candles (2023)'}, {'href': 'https://subslikescript.com/movie/Shaquille_Oneal_All-star_Comedy_Jam_Live_from_Sin_City-6833650', 'title': "Read transcript of Movie 'Shaquille O'neal All-star Comedy Jam: Live from Sin City'", 'movie_title': "Shaquille O'neal All-star Comedy Jam: Live from Sin 

In [114]:
# Pasar el json a un dataframe
movie_df = pd.DataFrame(all_movies)


In [116]:
# Ver info del df
movie_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   href         30 non-null     object
 1   title        30 non-null     object
 2   movie_title  30 non-null     object
dtypes: object(3)
memory usage: 848.0+ bytes


In [118]:
# Ahora con las 1911 páginas
def extract_movie_data(driver):
    movie_list = []
    movie_elements = driver.find_elements(By.CSS_SELECTOR, "ul.scripts-list li a")
    for movie_element in movie_elements:
        href = movie_element.get_attribute("href")
        title = movie_element.get_attribute("title")
        movie_title = movie_element.text.strip()
        movie_data = {
            "href": href,
            "title": title,
            "movie_title": movie_title
        }
        movie_list.append(movie_data)
    return movie_list

def scrape_all_pages(driver, base_url, total_pages):
    all_movie_data = []
    for page_number in range(1, total_pages + 1):
        url = f"{base_url}?page={page_number}"
        try:
            driver.get(url)
            wait = WebDriverWait(driver, 10)
            wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "ul.scripts-list li a")))
            movie_data = extract_movie_data(driver)
            all_movie_data.extend(movie_data)
            print(f"Scraped page {page_number}")  # Opcional: imprimir progreso
            time.sleep(1)  # Opcional: introduce un retraso
        except Exception as e:
            print(f"Error scraping page {page_number}: {e}")

    return all_movie_data

# Ejemplo de uso:
driver = web_driver()  # Reemplazar con el WebDriver deseado
base_url = "https://subslikescript.com/movies"
total_pages = 1911  # Reemplazar con el número total real de páginas

all_movies = scrape_all_pages(driver, base_url, total_pages)

# Convertir a DataFrame
movie_df = pd.DataFrame(all_movies)
print(movie_df)

driver.quit()


Scraped page 1
Scraped page 2
Scraped page 3
Scraped page 4
Scraped page 5
Scraped page 6
Scraped page 7
Scraped page 8
Scraped page 9
Scraped page 10
Scraped page 11
Scraped page 12
Scraped page 13
Scraped page 14
Scraped page 15
Scraped page 16
Scraped page 17
Scraped page 18
Scraped page 19
Scraped page 20
Scraped page 21
Scraped page 22
Scraped page 23
Scraped page 24
Scraped page 25
Scraped page 26
Scraped page 27
Scraped page 28
Scraped page 29
Scraped page 30
Scraped page 31
Scraped page 32
Scraped page 33
Scraped page 34
Scraped page 35
Scraped page 36
Scraped page 37
Scraped page 38
Scraped page 39
Scraped page 40
Scraped page 41
Scraped page 42
Scraped page 43
Scraped page 44
Scraped page 45
Scraped page 46
Scraped page 47
Scraped page 48
Scraped page 49
Scraped page 50
Scraped page 51
Scraped page 52
Scraped page 53
Scraped page 54
Scraped page 55
Scraped page 56
Scraped page 57
Scraped page 58
Scraped page 59
Scraped page 60
Scraped page 61
Scraped page 62
Scraped page 63
S

In [119]:
# Guardando el DataFrame como CSV 2 horas en scrapear, 57 319 registros
movie_df.to_csv("peliculas_scraped.csv", index=False)